In [121]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
current_directory = os.getcwd()
print(current_directory) 



/Users/srite/repos/sri_avi_blog_posts/share_buybacks


In [122]:
### uncomment the following line if you dont have yfinance installed
#!pip install yfinance
#!pip install chart_studio
import yfinance as yf

import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
#!pip install sec-api




In [123]:
pd.set_option('display.max_rows', 500)

# This Analysis will feature a 2 part analysis. 

## Part 1 -- do buyback announcements directly influence share prices?
#### Things to Look at
- 5, 30, 90 day share price avg pre and post annoucement
- 
- sdfkj

## Part 1

### Lets take a look at how 2023 and 2024 sharebuyback announcements immediately impacted share price by doing a pre vs post analysis

In [124]:
buybacks = pd.read_csv('buyback_announcements_2023_2024.csv')
buybacks.columns = buybacks.columns.str.replace(' ', '_')
buybacks.columns = buybacks.columns.str.replace('-', '_')

buybacks.head(5)

,Symbol,Company,Current_Price,Price_Change,Date,Percent_of_Shares,Buyback_Amount,Offer_Type,Buyback_Type,52_Week_Low,52_Week_High,Indicator(s)
0,ACN,Accenture,$371.27,2.90%,9/28/2023,2.00%,$4 billion,Open Market,Additional,$242.80,$375.73,Analyst Report - Analyst Revision - Gap Up -
1,ACM,AECOM,$89.23,1.30%,11/13/2023,8.30%,$1 billion,Open Market,NaN,$74.40,$94.01,Positive News -
2,A,Agilent Technologies,$132.01,-2.50%,1/9/2023,4.40%,$2 billion,Open Market,New,$96.80,$145.43,Upcoming Earnings - News Coverage - Gap Down -
3,ALKS,Alkermes,$28.91,-0.90%,2/15/2024,8.20%,$400 million,Open Market,New,$22.01,$33.71,Earnings Report - Analyst Downgrade - Analyst Revision -
4,MO,Altria Group,$40.90,1.10%,2/1/2024,1.40%,$1 billion,Open Market,New,$39.06,$48.04,Positive News -


In [125]:
#Get 5,30,90 day interval dates 
buybacks['Date'] = pd.to_datetime(buybacks['Date'])
buybacks['pre_date_5day'] = buybacks['Date']-timedelta(days=5)
buybacks['post_date_5day'] = buybacks['Date']+timedelta(days=5)
buybacks['pre_date_30day'] = buybacks['Date']-timedelta(days=30)
buybacks['post_date_30day'] = buybacks['Date']+timedelta(days=5)
buybacks['pre_date_90day'] = buybacks['Date']-timedelta(days=90)
buybacks['post_date_90day'] = buybacks['Date']+timedelta(days=90)


In [126]:
#Pull in share prices for dates in +- 90 days of Date using yfinance data


def fetch_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

stock_data = []

for index, row in buybacks.iterrows():
    ticker = row['Symbol']
    start_date = row['pre_date_90day']
    end_date = row['post_date_90day']
    announcement_date_price = row['Current_Price']
    buyback_date = row['Date']
    buyback_type = row['Buyback_Type']

    data_temp = fetch_stock_data(ticker, start_date, end_date)
    data_temp['Ticker'] = ticker
    data_temp['announcement_date'] = buyback_date;
    data_temp['announcement_date_price'] = announcement_date_price;
    data_temp['buyback_type'] = buyback_type;

    
    
    stock_data.append(data_temp)

    
stock_data = pd.concat(stock_data)
stock_data = stock_data.reset_index()



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [127]:
stock_data['announcement_date_price'] = stock_data['announcement_date_price'].str.replace('$','')
stock_data['announcement_date_price'] = stock_data['announcement_date_price'].str.replace(',','')

stock_data['announcement_date_price'] = stock_data['announcement_date_price'].astype(float)

In [128]:
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price,buyback_type
0,2023-06-30,309.489990,312.690002,308.329987,308.579987,305.080170,2668500,ACN,2023-09-28,371.27,Additional
1,2023-07-03,308.000000,311.269989,307.410004,310.970001,307.443054,1172800,ACN,2023-09-28,371.27,Additional
2,2023-07-05,306.140015,311.980011,305.029999,309.899994,306.385193,1924800,ACN,2023-09-28,371.27,Additional
3,2023-07-06,306.920013,308.450012,304.649994,307.970001,304.477081,1375800,ACN,2023-09-28,371.27,Additional
4,2023-07-07,306.730011,311.260010,305.279999,305.760010,302.292145,1902800,ACN,2023-09-28,371.27,Additional
...,...,...,...,...,...,...,...,...,...,...,...
10772,2023-07-18,18.750000,19.700001,18.660000,19.510000,19.309624,3444600,VNO,2023-04-26,25.53,NaN
10773,2023-07-19,19.860001,20.549999,19.790001,20.530001,20.319147,3945700,VNO,2023-04-26,25.53,NaN
10774,2023-07-20,20.150000,20.379999,19.520000,20.049999,19.844076,5440600,VNO,2023-04-26,25.53,NaN
10775,2023-07-21,20.420000,20.600000,19.830000,20.309999,20.101406,2527600,VNO,2023-04-26,25.53,NaN


In [129]:
# Get announcement date share price
announcement_dates = stock_data[['Ticker', 'announcement_date']].drop_duplicates()
 
announcement_dates = pd.merge(announcement_dates, stock_data[['Date','Close','Ticker']], left_on=['Ticker','announcement_date'] , right_on=['Ticker','Date'])
announcement_dates

announcement_dates.rename(columns={'Close':'announcement_date_price_v2'}, inplace=True)



stock_data_merged = pd.merge(stock_data, announcement_dates[['Ticker','announcement_date_price_v2']], left_on =['Ticker'], right_on =['Ticker'])

stock_data_merged
#announcement_dates

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price,buyback_type,announcement_date_price_v2
0,2023-06-30,309.489990,312.690002,308.329987,308.579987,305.080170,2668500,ACN,2023-09-28,371.27,Additional,300.769989
1,2023-07-03,308.000000,311.269989,307.410004,310.970001,307.443054,1172800,ACN,2023-09-28,371.27,Additional,300.769989
2,2023-07-05,306.140015,311.980011,305.029999,309.899994,306.385193,1924800,ACN,2023-09-28,371.27,Additional,300.769989
3,2023-07-06,306.920013,308.450012,304.649994,307.970001,304.477081,1375800,ACN,2023-09-28,371.27,Additional,300.769989
4,2023-07-07,306.730011,311.260010,305.279999,305.760010,302.292145,1902800,ACN,2023-09-28,371.27,Additional,300.769989
...,...,...,...,...,...,...,...,...,...,...,...,...
10649,2023-07-18,18.750000,19.700001,18.660000,19.510000,19.309624,3444600,VNO,2023-04-26,25.53,NaN,14.930000
10650,2023-07-19,19.860001,20.549999,19.790001,20.530001,20.319147,3945700,VNO,2023-04-26,25.53,NaN,14.930000
10651,2023-07-20,20.150000,20.379999,19.520000,20.049999,19.844076,5440600,VNO,2023-04-26,25.53,NaN,14.930000
10652,2023-07-21,20.420000,20.600000,19.830000,20.309999,20.101406,2527600,VNO,2023-04-26,25.53,NaN,14.930000


In [130]:
announcement_dates.head(5)

,Ticker,announcement_date,Date,announcement_date_price_v2
0,ACN,2023-09-28,2023-09-28,300.769989
1,ACM,2023-11-13,2023-11-13,80.470001
2,A,2023-01-09,2023-01-09,147.470001
3,ALKS,2024-02-15,2024-02-15,32.560001
4,MO,2024-02-01,2024-02-01,41.720001


In [131]:
stock_data_merged['deviation_from_announcement_date'] = (stock_data_merged['Close'] - stock_data_merged['announcement_date_price_v2'])
stock_data_merged['deviation_from_announcement_date_pct'] = round((stock_data_merged['Close'] - stock_data_merged['announcement_date_price_v2'])/stock_data_merged['announcement_date_price_v2'],3)



stock_data_merged['within_5_days'] = 0;
stock_data_merged['within_30_days'] = 0;
stock_data_merged['within_90_days'] = 0;


stock_data_merged.loc[abs( (stock_data_merged['Date'] - stock_data_merged['announcement_date']).dt.days) <= 5, 'within_5_days'] = 1
stock_data_merged.loc[abs( (stock_data_merged['Date'] - stock_data_merged['announcement_date']).dt.days) <= 30, 'within_30_days'] = 1
stock_data_merged.loc[abs( (stock_data_merged['Date'] - stock_data_merged['announcement_date']).dt.days) <= 90, 'within_90_days'] = 1
stock_data_merged['announcement_offset'] = (stock_data_merged['Date'] - stock_data_merged['announcement_date']).dt.days

In [132]:
stock_data_merged.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price,buyback_type,announcement_date_price_v2,deviation_from_announcement_date,deviation_from_announcement_date_pct,within_5_days,within_30_days,within_90_days,announcement_offset
0,2023-06-30,309.489990,312.690002,308.329987,308.579987,305.080170,2668500,ACN,2023-09-28,371.27,Additional,300.769989,7.809998,0.026,0,0,1,-90
1,2023-07-03,308.000000,311.269989,307.410004,310.970001,307.443054,1172800,ACN,2023-09-28,371.27,Additional,300.769989,10.200012,0.034,0,0,1,-87
2,2023-07-05,306.140015,311.980011,305.029999,309.899994,306.385193,1924800,ACN,2023-09-28,371.27,Additional,300.769989,9.130005,0.030,0,0,1,-85
3,2023-07-06,306.920013,308.450012,304.649994,307.970001,304.477081,1375800,ACN,2023-09-28,371.27,Additional,300.769989,7.200012,0.024,0,0,1,-84
4,2023-07-07,306.730011,311.260010,305.279999,305.760010,302.292145,1902800,ACN,2023-09-28,371.27,Additional,300.769989,4.990021,0.017,0,0,1,-83


In [133]:
stock_data_merged.loc[stock_data_merged['Ticker'] == 'NVR',]

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price,buyback_type,announcement_date_price_v2,deviation_from_announcement_date,deviation_from_announcement_date_pct,within_5_days,within_30_days,within_90_days,announcement_offset
6611,2023-11-16,6293.500000,6349.490234,6236.839844,6290.589844,6290.589844,25000,NVR,2024-02-14,7533.28,NaN,7501.72998,-1211.140137,-0.161,0,0,1,-90
6612,2023-11-17,6332.970215,6332.970215,6253.520020,6298.000000,6298.000000,23000,NVR,2024-02-14,7533.28,NaN,7501.72998,-1203.729980,-0.160,0,0,1,-89
6613,2023-11-20,6280.959961,6335.569824,6203.100098,6307.120117,6307.120117,22400,NVR,2024-02-14,7533.28,NaN,7501.72998,-1194.609863,-0.159,0,0,1,-86
6614,2023-11-21,6290.009766,6344.910156,6239.370117,6244.220215,6244.220215,20500,NVR,2024-02-14,7533.28,NaN,7501.72998,-1257.509766,-0.168,0,0,1,-85
6615,2023-11-22,6276.770020,6350.000000,6194.069824,6222.890137,6222.890137,12800,NVR,2024-02-14,7533.28,NaN,7501.72998,-1278.839844,-0.170,0,0,1,-84
6616,2023-11-24,6196.009766,6232.830078,6143.229980,6195.060059,6195.060059,6700,NVR,2024-02-14,7533.28,NaN,7501.72998,-1306.669922,-0.174,0,0,1,-82
6617,2023-11-27,6199.000000,6199.000000,6126.870117,6146.899902,6146.899902,14700,NVR,2024-02-14,7533.28,NaN,7501.72998,-1354.830078,-0.181,0,0,1,-79
6618,2023-11-28,6148.000000,6150.520020,6088.720215,6128.660156,6128.660156,24400,NVR,2024-02-14,7533.28,NaN,7501.72998,-1373.069824,-0.183,0,0,1,-78
6619,2023-11-29,6164.000000,6166.069824,6089.629883,6107.109863,6107.109863,22700,NVR,2024-02-14,7533.28,NaN,7501.72998,-1394.620117,-0.186,0,0,1,-77
6620,2023-11-30,6107.109863,6159.540039,6052.580078,6155.390137,6155.390137,29900,NVR,2024-02-14,7533.28,NaN,7501.72998,-1346.339844,-0.179,0,0,1,-76


In [134]:
# 30 day chart

stock_data_30day = stock_data_merged.loc[stock_data_merged['within_30_days'] == 1,]
stock_data_30day.head(50)
fig = px.line(stock_data_30day, x="announcement_offset", y="deviation_from_announcement_date_pct", color='Ticker')
fig.show()


In [135]:
## Add 30 day averages
#avg_30day = stock_data_30day
avg_30day_grouped = stock_data_30day[['Ticker', 'announcement_offset','deviation_from_announcement_date_pct']].groupby('announcement_offset')['deviation_from_announcement_date_pct'].mean()
avg_30day_grouped = avg_30day_grouped.to_frame()
avg_30day_grouped = avg_30day_grouped.reset_index()




In [136]:
# 30 day chart WITH average chart

stock_data_30day = stock_data_merged.loc[stock_data_merged['within_30_days'] == 1,]
stock_data_30day.head(50)
fig = px.line(stock_data_30day, x="announcement_offset", y="deviation_from_announcement_date_pct", color='Ticker',
             labels={
                     "announcement_offset": "Days from Buyback Announcement",
                     "deviation_from_announcement_date_pct": "% deviation from price on announcement date"
                 })
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})




## Add 30 day averages
#avg_30day = stock_data_30day
avg_30day_grouped = stock_data_30day[['Ticker', 'announcement_offset','deviation_from_announcement_date_pct']].groupby('announcement_offset')['deviation_from_announcement_date_pct'].mean()
avg_30day_grouped = avg_30day_grouped.to_frame()
avg_30day_grouped = avg_30day_grouped.reset_index()


## Add Chart element
trace1 = go.Scatter(
    x=avg_30day_grouped['announcement_offset'].tolist(),
    y=avg_30day_grouped['deviation_from_announcement_date_pct'].tolist(),
    line=dict(
        color='black',
        width = 5),
    name='Average'
    #xperiod="M1"
    #xperiodalignment="middle"
    #hovertemplate="%{y}%{_xother}"
)




fig.add_trace(trace1)


In [137]:
go.Figure(data=[trace1]).show()


In [138]:
# 5 day chart

stock_data_5day = stock_data_merged.loc[stock_data_merged['within_5_days'] == 1,]
fig = px.line(stock_data_5day, x="announcement_offset", y="deviation_from_announcement_date_pct", color='Ticker',
             labels={
                     "announcement_offset": "Days from Buyback Announcement",
                     "deviation_from_announcement_date_pct": "% deviation from price on announcement date"
                 },
)
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})

fig.show()


In [139]:
## Add 5 day averages
#avg_30day = stock_data_30day
avg_5day_grouped = stock_data_5day[['Ticker', 'announcement_offset','deviation_from_announcement_date_pct']].groupby('announcement_offset')['deviation_from_announcement_date_pct'].mean()
avg_5day_grouped = avg_5day_grouped.to_frame()
avg_5day_grouped = avg_5day_grouped.reset_index()




In [140]:

## Add Chart element
trace1 = go.Scatter(
    x=avg_5day_grouped['announcement_offset'].tolist(),
    y=avg_5day_grouped['deviation_from_announcement_date_pct'].tolist(),
    line=dict(
        color='black',
        width = 5),

    name='Average'
    #xperiod="M1"
    #xperiodalignment="middle"
    #hovertemplate="%{y}%{_xother}"
)




fig.add_trace(trace1)


In [141]:
stock_data_merged.loc[stock_data_merged['Ticker'] == 'EXPE',]

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price,buyback_type,announcement_date_price_v2,deviation_from_announcement_date,deviation_from_announcement_date_pct,within_5_days,within_30_days,within_90_days,announcement_offset
2748,2023-08-04,101.089996,104.230003,101.010002,103.190002,103.190002,5801200,EXPE,2023-11-02,139.1,NaN,94.839996,8.350006,0.088,0,0,1,-90
2749,2023-08-07,104.070000,108.489998,104.029999,107.489998,107.489998,5171500,EXPE,2023-11-02,139.1,NaN,94.839996,12.650002,0.133,0,0,1,-87
2750,2023-08-08,106.870003,110.339996,105.570000,109.839996,109.839996,3014100,EXPE,2023-11-02,139.1,NaN,94.839996,15.000000,0.158,0,0,1,-86
2751,2023-08-09,110.309998,111.970001,109.529999,109.809998,109.809998,3156300,EXPE,2023-11-02,139.1,NaN,94.839996,14.970001,0.158,0,0,1,-85
2752,2023-08-10,111.010002,112.570000,110.379997,111.669998,111.669998,2268600,EXPE,2023-11-02,139.1,NaN,94.839996,16.830002,0.177,0,0,1,-84
2753,2023-08-11,110.459999,112.470001,110.459999,111.540001,111.540001,2840500,EXPE,2023-11-02,139.1,NaN,94.839996,16.700005,0.176,0,0,1,-83
2754,2023-08-14,111.010002,112.059998,109.309998,111.930000,111.930000,1795700,EXPE,2023-11-02,139.1,NaN,94.839996,17.090004,0.180,0,0,1,-80
2755,2023-08-15,111.980003,112.330002,106.660004,107.430000,107.430000,3225700,EXPE,2023-11-02,139.1,NaN,94.839996,12.590004,0.133,0,0,1,-79
2756,2023-08-16,107.050003,108.360001,106.790001,107.139999,107.139999,1989200,EXPE,2023-11-02,139.1,NaN,94.839996,12.300003,0.130,0,0,1,-78
2757,2023-08-17,108.000000,108.000000,105.580002,105.629997,105.629997,3045800,EXPE,2023-11-02,139.1,NaN,94.839996,10.790001,0.114,0,0,1,-77


### Filtering on only new buyback announcements

In [142]:
new_announcements = stock_data_merged.loc[stock_data_merged['buyback_type'] == 'New',]

stock_data_30day_newannouncements = new_announcements.loc[new_announcements['within_30_days'] == 1,]
stock_data_30day_newannouncements.head(50)
fig = px.line(stock_data_30day_newannouncements, x="announcement_offset", y="deviation_from_announcement_date_pct", color='Ticker',
             labels={
                     "announcement_offset": "Days from Buyback Announcement",
                     "deviation_from_announcement_date_pct": "% deviation from price on announcement date"
                 },)


fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})


fig.show()

In [143]:
## Add 30 day averages
#avg_30day = stock_data_30day
avg_30day_grouped_na = stock_data_30day_newannouncements[['Ticker', 'announcement_offset','deviation_from_announcement_date_pct']].groupby('announcement_offset')['deviation_from_announcement_date_pct'].mean()
avg_30day_grouped_na = avg_30day_grouped_na.to_frame()
avg_30day_grouped_na = avg_30day_grouped_na.reset_index()


## Add Chart element
trace1 = go.Scatter(
    x=avg_30day_grouped_na['announcement_offset'].tolist(),
    y=avg_30day_grouped_na['deviation_from_announcement_date_pct'].tolist(),
    line=dict(
        color='black',
        width = 5),
    name='Average'
    #xperiod="M1"
    #xperiodalignment="middle"
    #hovertemplate="%{y}%{_xother}"
)




fig.add_trace(trace1)


In [144]:
go.Figure(data=[trace1]).show()

# Looking at actual buying back of share price on volume

## What do I want to look at?

- For companies with buyback announcements, what does share count look like pre and post announcement?
- How does share price correlate with downward trend of share count
- If possible, can we compare this against share price of companies that have increasing share counts?

# Appendix and sandbox

In [145]:
## read in yfinance data for stocks

aapl = yf.Ticker('AAPL')
aapl_df = aapl.history(period='1y')
#aapl_df['Close'].plot(title="APPLE's stock")
aapl_df = aapl_df.reset_index()
aapl_df



,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-02-27 00:00:00-05:00,146.925767,148.378007,146.667138,147.134644,44998500,0.00,0.0
1,2023-02-28 00:00:00-05:00,146.269261,148.288481,146.050427,146.627350,50547000,0.00,0.0
2,2023-03-01 00:00:00-05:00,146.050447,146.448318,144.240103,144.538513,55479000,0.00,0.0
3,2023-03-02 00:00:00-05:00,143.613455,145.931086,143.135992,145.135330,52238100,0.00,0.0
4,2023-03-03 00:00:00-05:00,147.254004,150.307711,146.547782,150.228134,70732300,0.00,0.0
5,2023-03-06 00:00:00-05:00,152.973479,155.470162,152.645244,153.013275,87558000,0.00,0.0
6,2023-03-07 00:00:00-05:00,152.883946,153.212196,150.327599,150.795105,56182000,0.00,0.0
7,2023-03-08 00:00:00-05:00,151.998671,152.655170,151.023878,152.058350,47204800,0.00,0.0
8,2023-03-09 00:00:00-05:00,152.744699,153.719491,149.432377,149.790466,53833600,0.00,0.0
9,2023-03-10 00:00:00-05:00,149.412475,150.138595,146.826274,147.711548,68572400,0.00,0.0


In [147]:
from sec_api import QueryApi, FloatApi

floatApi = FloatApi(api_key="1136c7de8da814bb09eea22d0cc6768d1493907372dbf5d88faf3bcd5e419422")
response = floatApi.get_float(ticker='GOOG')

df = pd.json_normalize(response['data'])
df['reportedAt'] = pd.to_datetime(df['reportedAt'])
df.drop(columns=['id'], inplace=True) # 'tickers', 'cik'
df.sort_values(by='periodOfReport')


# float.outstandingShares
df2 = df.explode('float.outstandingShares')['float.outstandingShares'].apply(pd.Series)
df2 = df2.rename(columns={"period": "outstandingShares.period", 
                          "shareClass": "outstandingShares.shareClass", 
                          "value": "outstandingShares.value"})

# float.publicFloat
df3 = df.explode('float.publicFloat')['float.publicFloat'].apply(pd.Series)
df3 = df3.rename(columns={"period": "publicFloat.period", 
                          "shareClass": "publicFloat.shareClass", 
                          "value": "publicFloat.value"})

df4 = df.copy()
#df4['totalSharecount']

float_data = df.join([df2, df3], how='outer').drop(columns=['float.outstandingShares', 'float.publicFloat', 'sourceFilingAccessionNo'])

float_data

,tickers,cik,reportedAt,periodOfReport,outstandingShares.period,outstandingShares.shareClass,outstandingShares.value,publicFloat.period,publicFloat.shareClass,publicFloat.value,0
0,"[GOOGL, GOOG]",1652044,2024-01-30 21:43:43-05:00,2023-12-31,2024-01-23,CommonClassA,5893000000,2023-06-30,,1.331200e+12,NaN
0,"[GOOGL, GOOG]",1652044,2024-01-30 21:43:43-05:00,2023-12-31,2024-01-23,CommonClassB,869000000,2023-06-30,,1.331200e+12,NaN
0,"[GOOGL, GOOG]",1652044,2024-01-30 21:43:43-05:00,2023-12-31,2024-01-23,CapitalClassC,5671000000,2023-06-30,,1.331200e+12,NaN
1,"[GOOGL, GOOG]",1652044,2023-10-24 18:39:13-04:00,2023-09-30,2023-10-17,CommonClassA,5918000000,NaN,NaN,NaN,NaN
1,"[GOOGL, GOOG]",1652044,2023-10-24 18:39:13-04:00,2023-09-30,2023-10-17,CommonClassB,873000000,NaN,NaN,NaN,NaN
1,"[GOOGL, GOOG]",1652044,2023-10-24 18:39:13-04:00,2023-09-30,2023-10-17,CapitalClassC,5725000000,NaN,NaN,NaN,NaN
2,"[GOOGL, GOOG]",1652044,2023-07-25 19:13:26-04:00,2023-06-30,2023-07-18,CommonClassA,5933000000,NaN,NaN,NaN,NaN
2,"[GOOGL, GOOG]",1652044,2023-07-25 19:13:26-04:00,2023-06-30,2023-07-18,CommonClassB,875000000,NaN,NaN,NaN,NaN
2,"[GOOGL, GOOG]",1652044,2023-07-25 19:13:26-04:00,2023-06-30,2023-07-18,CapitalClassC,5801000000,NaN,NaN,NaN,NaN
3,"[GOOGL, GOOG]",1652044,2023-04-25 20:15:00-04:00,2023-03-31,2023-04-18,CommonClassA,5941000000,NaN,NaN,NaN,NaN


In [148]:
import json

#create function that iterates through dictionary and returns sum of shares for that dictionary
def sum_values(dictionary):
    total_sum = 0
    for item in dictionary:
        total_sum += item['value']
    return total_sum;

df['totalShares'] = df['float.outstandingShares'].apply(sum_values)
df.head()

,tickers,cik,reportedAt,periodOfReport,sourceFilingAccessionNo,float.outstandingShares,float.publicFloat,totalShares
0,"[GOOGL, GOOG]",1652044,2024-01-30 21:43:43-05:00,2023-12-31,0001652044-24-000022,"[{'period': '2024-01-23', 'shareClass': 'CommonClassA', 'value': 5893000000}, {'period': '2024-01-23', 'shareClass': 'CommonClassB', 'value': 869000000}, {'period': '2024-01-23', 'shareClass': 'CapitalClassC', 'value': 5671000000}]","[{'period': '2023-06-30', 'shareClass': '', 'value': 1331200000000}]",12433000000
1,"[GOOGL, GOOG]",1652044,2023-10-24 18:39:13-04:00,2023-09-30,0001652044-23-000094,"[{'period': '2023-10-17', 'shareClass': 'CommonClassA', 'value': 5918000000}, {'period': '2023-10-17', 'shareClass': 'CommonClassB', 'value': 873000000}, {'period': '2023-10-17', 'shareClass': 'CapitalClassC', 'value': 5725000000}]",[],12516000000
2,"[GOOGL, GOOG]",1652044,2023-07-25 19:13:26-04:00,2023-06-30,0001652044-23-000070,"[{'period': '2023-07-18', 'shareClass': 'CommonClassA', 'value': 5933000000}, {'period': '2023-07-18', 'shareClass': 'CommonClassB', 'value': 875000000}, {'period': '2023-07-18', 'shareClass': 'CapitalClassC', 'value': 5801000000}]",[],12609000000
3,"[GOOGL, GOOG]",1652044,2023-04-25 20:15:00-04:00,2023-03-31,0001652044-23-000045,"[{'period': '2023-04-18', 'shareClass': 'CommonClassA', 'value': 5941000000}, {'period': '2023-04-18', 'shareClass': 'CommonClassB', 'value': 882000000}, {'period': '2023-04-18', 'shareClass': 'CapitalClassC', 'value': 5874000000}]",[],12697000000
4,"[GOOGL, GOOG]",1652044,2023-02-02 21:23:45-05:00,2022-12-31,0001652044-23-000016,"[{'period': '2023-01-26', 'shareClass': 'CommonClassA', 'value': 5956000000}, {'period': '2023-01-26', 'shareClass': 'CommonClassB', 'value': 883000000}, {'period': '2023-01-26', 'shareClass': 'CapitalClassC', 'value': 5968000000}]","[{'period': '2022-06-30', 'shareClass': '', 'value': 1256100000000}]",12807000000


In [149]:
### Dataset structure ###


######## Table 1 - Company lookup - 1 row per company ########
######## Ticker | Buyback Announcement Date | Buyback Amount | First Day of Buyback Announcement Quarter | Buyback Quarter - 4 Quarters | Buyback Quarter + 4 quarters ########

######## Table 2 - Share count and price tracker - 8 row per company, one for each quarter (4 pre and 4 post) ########
######## Ticker | Buyback Announcement Date | Buyback Amount | Quarter | Sharecount | Avg Weighted Price ########


In [158]:
##### Creating Table 1

from pandas.tseries.offsets import DateOffset
buybacks['Date'] = pd.to_datetime(buybacks['Date'])

shares_company_lookup = buybacks.copy()
shares_company_lookup = shares_company_lookup.loc[shares_company_lookup['Buyback_Type'] == 'New',]
#shares_company_lookup['buyback_announcement_quarter'] = shares_company_lookup['Date'].to_period('Q').to_timestamp()
shares_company_lookup['buyback_announcement_quarter'] = pd.PeriodIndex(shares_company_lookup['Date'], freq='Q').to_timestamp()
shares_company_lookup['announcement_4q_pre']  = shares_company_lookup['buyback_announcement_quarter'] + DateOffset(months=-12)
shares_company_lookup['announcement_4q_post']  = shares_company_lookup['buyback_announcement_quarter'] + DateOffset(months=12)

#buybacks['pre_date_5day'] = buybacks['Date']-timedelta(days=5)

shares_company_lookup.head()


,Symbol,Company,Current_Price,Price_Change,Date,Percent_of_Shares,Buyback_Amount,Offer_Type,Buyback_Type,52_Week_Low,...,Indicator(s),pre_date_5day,post_date_5day,pre_date_30day,post_date_30day,pre_date_90day,post_date_90day,buyback_announcement_quarter,announcement_4q_pre,announcement_4q_post
2,A,Agilent Technologies,$132.01,-2.50%,2023-01-09,4.40%,$2 billion,Open Market,New,$96.80,...,Upcoming Earnings - News Coverage - Gap Down -,2023-01-04,2023-01-14,2022-12-10,2023-01-14,2022-10-11,2023-04-09,2023-01-01,2022-01-01,2024-01-01
3,ALKS,Alkermes,$28.91,-0.90%,2024-02-15,8.20%,$400 million,Open Market,New,$22.01,...,Earnings Report - Analyst Downgrade - Analyst Revision -,2024-02-10,2024-02-20,2024-01-16,2024-02-20,2023-11-17,2024-05-15,2024-01-01,2023-01-01,2025-01-01
4,MO,Altria Group,$40.90,1.10%,2024-02-01,1.40%,$1 billion,Open Market,New,$39.06,...,Positive News -,2024-01-27,2024-02-06,2024-01-02,2024-02-06,2023-11-03,2024-05-01,2024-01-01,2023-01-01,2025-01-01
10,ASH,Ashland,$93.02,0.90%,2023-06-28,21.20%,$1 billion,Open Market,New,$70.82,...,NaN,2023-06-23,2023-07-03,2023-05-29,2023-07-03,2023-03-30,2023-09-26,2023-04-01,2022-04-01,2024-04-01
12,ATI,ATI,$46.46,1.30%,2023-11-29,2.90%,$150 million,Open Market,New,$34.10,...,Analyst Report -,2023-11-24,2023-12-04,2023-10-30,2023-12-04,2023-08-31,2024-02-27,2023-10-01,2022-10-01,2024-10-01


In [159]:
shares_company_lookup.head()

,Symbol,Company,Current_Price,Price_Change,Date,Percent_of_Shares,Buyback_Amount,Offer_Type,Buyback_Type,52_Week_Low,...,Indicator(s),pre_date_5day,post_date_5day,pre_date_30day,post_date_30day,pre_date_90day,post_date_90day,buyback_announcement_quarter,announcement_4q_pre,announcement_4q_post
2,A,Agilent Technologies,$132.01,-2.50%,2023-01-09,4.40%,$2 billion,Open Market,New,$96.80,...,Upcoming Earnings - News Coverage - Gap Down -,2023-01-04,2023-01-14,2022-12-10,2023-01-14,2022-10-11,2023-04-09,2023-01-01,2022-01-01,2024-01-01
3,ALKS,Alkermes,$28.91,-0.90%,2024-02-15,8.20%,$400 million,Open Market,New,$22.01,...,Earnings Report - Analyst Downgrade - Analyst Revision -,2024-02-10,2024-02-20,2024-01-16,2024-02-20,2023-11-17,2024-05-15,2024-01-01,2023-01-01,2025-01-01
4,MO,Altria Group,$40.90,1.10%,2024-02-01,1.40%,$1 billion,Open Market,New,$39.06,...,Positive News -,2024-01-27,2024-02-06,2024-01-02,2024-02-06,2023-11-03,2024-05-01,2024-01-01,2023-01-01,2025-01-01
10,ASH,Ashland,$93.02,0.90%,2023-06-28,21.20%,$1 billion,Open Market,New,$70.82,...,NaN,2023-06-23,2023-07-03,2023-05-29,2023-07-03,2023-03-30,2023-09-26,2023-04-01,2022-04-01,2024-04-01
12,ATI,ATI,$46.46,1.30%,2023-11-29,2.90%,$150 million,Open Market,New,$34.10,...,Analyst Report -,2023-11-24,2023-12-04,2023-10-30,2023-12-04,2023-08-31,2024-02-27,2023-10-01,2022-10-01,2024-10-01


In [161]:
def quarter_list(beg, end):
    quarter_list = []
    quarter_list.append(beg)
    curr_quarter = beg + DateOffset(months=4)
    quarter_list.append(curr_quarter)

    for i in range(10):
        curr_quarter = curr_quarter + DateOffset(months=4)
        if curr_quarter <= end:
            quarter_list.append(curr_quarter)
        else:
            break
    return quarter_list

In [162]:
shares_company_lookup['quarter'] = shares_company_lookup.apply(lambda x: quarter_list(x['announcement_4q_pre'], x['announcement_4q_post']), axis=1)
shares_company_lookup.head()

,Symbol,Company,Current_Price,Price_Change,Date,Percent_of_Shares,Buyback_Amount,Offer_Type,Buyback_Type,52_Week_Low,...,pre_date_5day,post_date_5day,pre_date_30day,post_date_30day,pre_date_90day,post_date_90day,buyback_announcement_quarter,announcement_4q_pre,announcement_4q_post,quarter
2,A,Agilent Technologies,$132.01,-2.50%,2023-01-09,4.40%,$2 billion,Open Market,New,$96.80,...,2023-01-04,2023-01-14,2022-12-10,2023-01-14,2022-10-11,2023-04-09,2023-01-01,2022-01-01,2024-01-01,"[2022-01-01 00:00:00, 2022-05-01 00:00:00, 2022-09-01 00:00:00, 2023-01-01 00:00:00, 2023-05-01 00:00:00, 2023-09-01 00:00:00, 2024-01-01 00:00:00]"
3,ALKS,Alkermes,$28.91,-0.90%,2024-02-15,8.20%,$400 million,Open Market,New,$22.01,...,2024-02-10,2024-02-20,2024-01-16,2024-02-20,2023-11-17,2024-05-15,2024-01-01,2023-01-01,2025-01-01,"[2023-01-01 00:00:00, 2023-05-01 00:00:00, 2023-09-01 00:00:00, 2024-01-01 00:00:00, 2024-05-01 00:00:00, 2024-09-01 00:00:00, 2025-01-01 00:00:00]"
4,MO,Altria Group,$40.90,1.10%,2024-02-01,1.40%,$1 billion,Open Market,New,$39.06,...,2024-01-27,2024-02-06,2024-01-02,2024-02-06,2023-11-03,2024-05-01,2024-01-01,2023-01-01,2025-01-01,"[2023-01-01 00:00:00, 2023-05-01 00:00:00, 2023-09-01 00:00:00, 2024-01-01 00:00:00, 2024-05-01 00:00:00, 2024-09-01 00:00:00, 2025-01-01 00:00:00]"
10,ASH,Ashland,$93.02,0.90%,2023-06-28,21.20%,$1 billion,Open Market,New,$70.82,...,2023-06-23,2023-07-03,2023-05-29,2023-07-03,2023-03-30,2023-09-26,2023-04-01,2022-04-01,2024-04-01,"[2022-04-01 00:00:00, 2022-08-01 00:00:00, 2022-12-01 00:00:00, 2023-04-01 00:00:00, 2023-08-01 00:00:00, 2023-12-01 00:00:00, 2024-04-01 00:00:00]"
12,ATI,ATI,$46.46,1.30%,2023-11-29,2.90%,$150 million,Open Market,New,$34.10,...,2023-11-24,2023-12-04,2023-10-30,2023-12-04,2023-08-31,2024-02-27,2023-10-01,2022-10-01,2024-10-01,"[2022-10-01 00:00:00, 2023-02-01 00:00:00, 2023-06-01 00:00:00, 2023-10-01 00:00:00, 2024-02-01 00:00:00, 2024-06-01 00:00:00, 2024-10-01 00:00:00]"


In [163]:
shares_company_lookup_short = shares_company_lookup[['Symbol','Company', 'Date','Percent_of_Shares', 'Buyback_Amount', 'quarter']]
shares_company_lookup_short = shares_company_lookup_short.explode('quarter')
shares_company_lookup_short = shares_company_lookup_short.reset_index(drop=True)
shares_company_lookup_short.head(10)

,Symbol,Company,Date,Percent_of_Shares,Buyback_Amount,quarter
0,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2022-01-01
1,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2022-05-01
2,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2022-09-01
3,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2023-01-01
4,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2023-05-01
5,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2023-09-01
6,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2024-01-01
7,ALKS,Alkermes,2024-02-15,8.20%,$400 million,2023-01-01
8,ALKS,Alkermes,2024-02-15,8.20%,$400 million,2023-05-01
9,ALKS,Alkermes,2024-02-15,8.20%,$400 million,2023-09-01


In [164]:
def getshareprice(ticker, date):
    ticker_data = yf.download(ticker, start = date + DateOffset(months=-3), end=date)
    
    return ticker_data['Close'].mean()
    
    
    
shares_company_lookup_short['share_price_30day_rolling'] = shares_company_lookup_short.apply(lambda x: getshareprice(x['Symbol'], x['quarter']), axis=1)
shares_company_lookup_short.dtypes

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ALKS']: Exception("%ticker%: Data doesn't exist for startDate = 1717214400, endDa

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['MNST']: Exce

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['URI']: Exception("%ticker%: Data doesn't exist for startDate = 1717214400, endDate = 1725163200")
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['URI']: Exception("%ticker%: Data doesn't exist for startDate = 1727755200, endDate = 1735707600")


Symbol                               object
Company                              object
Date                         datetime64[ns]
Percent_of_Shares                    object
Buyback_Amount                       object
quarter                      datetime64[ns]
share_price_30day_rolling           float64
dtype: object

In [165]:
shares_company_lookup_short.loc[shares_company_lookup_short['Symbol'] == 'A',]

,Symbol,Company,Date,Percent_of_Shares,Buyback_Amount,quarter,share_price_30day_rolling
0,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2022-01-01,155.773594
1,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2022-05-01,132.556612
2,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2022-09-01,125.647656
3,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2023-01-01,142.774127
4,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2023-05-01,140.885901
5,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2023-09-01,121.184844
6,A,Agilent Technologies,2023-01-09,4.40%,$2 billion,2024-01-01,118.970476


In [ ]:
#### Now lets stitch in share counts

In [194]:
floatApi = FloatApi(api_key="260d219ce50cec67c24341f80b1eff13055181eea26a91554faa54d270ecac76")

def sum_value(dictionary):
    total_sum = 0
    for item in dictionary:
        total_sum += item['value']
    return total_sum;




def querySecData(ticker_name, date):
    response1 = floatApi.get_float(ticker=ticker_name)
    response1 = pd.json_normalize(response1['data'])
    response1['reportedAt'] = pd.to_datetime(response1['reportedAt'])
    response1.drop(columns=['id'], inplace=True) # 'tickers', 'cik'
    response1.sort_values(by='periodOfReport')

    #response1 = response1.loc[response1['periodOfReport']== date,]
    response1['totalShares'] = response1['float.outstandingShares'].apply(sum_values)
    return response1['totalShares'][0]
    


shares_company_lookup_short['shareCount'] = shares_company_lookup_short.apply(lambda x: querySecData(x['Symbol'], x['quarter']), axis=1)
shares_company_lookup_short

#df1 = response1.loc[response1['reportedAt']]

   tickers      cik                 reportedAt periodOfReport  \
0      [A]  1090872  2023-12-19 18:52:16-05:00     2023-10-31   
1      [A]  1090872  2023-08-30 18:47:03-04:00     2023-07-31   
2      [A]  1090872  2023-05-26 17:28:42-04:00     2023-04-30   
3      [A]  1090872  2023-03-02 18:52:33-05:00     2023-01-31   
4      [A]  1090872  2022-12-20 18:42:30-05:00     2022-10-31   
5      [A]  1090872  2022-08-31 18:53:26-04:00     2022-07-31   
6      [A]  1090872  2022-05-27 19:33:57-04:00     2022-04-30   
7      [A]  1090872  2022-03-03 17:02:17-05:00     2022-01-31   
8      [A]  1090872  2021-12-17 17:01:49-05:00     2021-10-31   
9      [A]  1090872  2021-08-31 19:01:13-04:00     2021-07-31   
10     [A]  1090872  2021-05-28 18:43:28-04:00     2021-04-30   
11     [A]  1090872  2021-03-02 16:44:15-05:00     2021-01-31   
12     [A]  1090872  2020-12-17 19:56:21-05:00     2020-10-31   
13     [A]  1090872  2020-09-01 16:41:24-04:00     2020-07-31   
14     [A]  1090872  2020

   tickers      cik                 reportedAt periodOfReport  \
0      [A]  1090872  2023-12-19 18:52:16-05:00     2023-10-31   
1      [A]  1090872  2023-08-30 18:47:03-04:00     2023-07-31   
2      [A]  1090872  2023-05-26 17:28:42-04:00     2023-04-30   
3      [A]  1090872  2023-03-02 18:52:33-05:00     2023-01-31   
4      [A]  1090872  2022-12-20 18:42:30-05:00     2022-10-31   
5      [A]  1090872  2022-08-31 18:53:26-04:00     2022-07-31   
6      [A]  1090872  2022-05-27 19:33:57-04:00     2022-04-30   
7      [A]  1090872  2022-03-03 17:02:17-05:00     2022-01-31   
8      [A]  1090872  2021-12-17 17:01:49-05:00     2021-10-31   
9      [A]  1090872  2021-08-31 19:01:13-04:00     2021-07-31   
10     [A]  1090872  2021-05-28 18:43:28-04:00     2021-04-30   
11     [A]  1090872  2021-03-02 16:44:15-05:00     2021-01-31   
12     [A]  1090872  2020-12-17 19:56:21-05:00     2020-10-31   
13     [A]  1090872  2020-09-01 16:41:24-04:00     2020-07-31   
14     [A]  1090872  2020

   tickers      cik                 reportedAt periodOfReport  \
0      [A]  1090872  2023-12-19 18:52:16-05:00     2023-10-31   
1      [A]  1090872  2023-08-30 18:47:03-04:00     2023-07-31   
2      [A]  1090872  2023-05-26 17:28:42-04:00     2023-04-30   
3      [A]  1090872  2023-03-02 18:52:33-05:00     2023-01-31   
4      [A]  1090872  2022-12-20 18:42:30-05:00     2022-10-31   
5      [A]  1090872  2022-08-31 18:53:26-04:00     2022-07-31   
6      [A]  1090872  2022-05-27 19:33:57-04:00     2022-04-30   
7      [A]  1090872  2022-03-03 17:02:17-05:00     2022-01-31   
8      [A]  1090872  2021-12-17 17:01:49-05:00     2021-10-31   
9      [A]  1090872  2021-08-31 19:01:13-04:00     2021-07-31   
10     [A]  1090872  2021-05-28 18:43:28-04:00     2021-04-30   
11     [A]  1090872  2021-03-02 16:44:15-05:00     2021-01-31   
12     [A]  1090872  2020-12-17 19:56:21-05:00     2020-10-31   
13     [A]  1090872  2020-09-01 16:41:24-04:00     2020-07-31   
14     [A]  1090872  2020

   tickers      cik                 reportedAt periodOfReport  \
0      [A]  1090872  2023-12-19 18:52:16-05:00     2023-10-31   
1      [A]  1090872  2023-08-30 18:47:03-04:00     2023-07-31   
2      [A]  1090872  2023-05-26 17:28:42-04:00     2023-04-30   
3      [A]  1090872  2023-03-02 18:52:33-05:00     2023-01-31   
4      [A]  1090872  2022-12-20 18:42:30-05:00     2022-10-31   
5      [A]  1090872  2022-08-31 18:53:26-04:00     2022-07-31   
6      [A]  1090872  2022-05-27 19:33:57-04:00     2022-04-30   
7      [A]  1090872  2022-03-03 17:02:17-05:00     2022-01-31   
8      [A]  1090872  2021-12-17 17:01:49-05:00     2021-10-31   
9      [A]  1090872  2021-08-31 19:01:13-04:00     2021-07-31   
10     [A]  1090872  2021-05-28 18:43:28-04:00     2021-04-30   
11     [A]  1090872  2021-03-02 16:44:15-05:00     2021-01-31   
12     [A]  1090872  2020-12-17 19:56:21-05:00     2020-10-31   
13     [A]  1090872  2020-09-01 16:41:24-04:00     2020-07-31   
14     [A]  1090872  2020

   tickers      cik                 reportedAt periodOfReport  \
0      [A]  1090872  2023-12-19 18:52:16-05:00     2023-10-31   
1      [A]  1090872  2023-08-30 18:47:03-04:00     2023-07-31   
2      [A]  1090872  2023-05-26 17:28:42-04:00     2023-04-30   
3      [A]  1090872  2023-03-02 18:52:33-05:00     2023-01-31   
4      [A]  1090872  2022-12-20 18:42:30-05:00     2022-10-31   
5      [A]  1090872  2022-08-31 18:53:26-04:00     2022-07-31   
6      [A]  1090872  2022-05-27 19:33:57-04:00     2022-04-30   
7      [A]  1090872  2022-03-03 17:02:17-05:00     2022-01-31   
8      [A]  1090872  2021-12-17 17:01:49-05:00     2021-10-31   
9      [A]  1090872  2021-08-31 19:01:13-04:00     2021-07-31   
10     [A]  1090872  2021-05-28 18:43:28-04:00     2021-04-30   
11     [A]  1090872  2021-03-02 16:44:15-05:00     2021-01-31   
12     [A]  1090872  2020-12-17 19:56:21-05:00     2020-10-31   
13     [A]  1090872  2020-09-01 16:41:24-04:00     2020-07-31   
14     [A]  1090872  2020

...

                             tickers      cik                 reportedAt  \
0   [MET, MET PRA, MET PRE, MET PRF]  1099219  2024-02-15 18:42:43-05:00   
1   [MET, MET PRA, MET PRE, MET PRF]  1099219  2023-11-02 16:47:52-04:00   
2   [MET, MET PRA, MET PRE, MET PRF]  1099219  2023-08-03 17:36:59-04:00   
3   [MET, MET PRA, MET PRE, MET PRF]  1099219  2023-05-04 16:58:19-04:00   
4   [MET, MET PRA, MET PRE, MET PRF]  1099219  2023-02-23 17:11:40-05:00   
5   [MET, MET PRA, MET PRE, MET PRF]  1099219  2022-11-03 17:21:33-04:00   
6   [MET, MET PRA, MET PRE, MET PRF]  1099219  2022-08-04 17:04:46-04:00   
7   [MET, MET PRA, MET PRE, MET PRF]  1099219  2022-05-05 16:33:59-04:00   
8   [MET, MET PRA, MET PRE, MET PRF]  1099219  2022-02-17 19:16:46-05:00   
9   [MET, MET PRA, MET PRE, MET PRF]  1099219  2021-11-08 18:42:57-05:00   
10  [MET, MET PRA, MET PRE, MET PRF]  1099219  2021-08-05 19:03:45-04:00   
11  [MET, MET PRA, MET PRE, MET PRF]  1099219  2021-05-06 17:24:25-04:00   
12  [MET, ME

   tickers     cik                 reportedAt periodOfReport  \
0    [MGM]  789570  2024-02-23 08:28:47-05:00     2023-12-31   
1    [MGM]  789570  2023-11-08 16:20:52-05:00     2023-09-30   
2    [MGM]  789570  2023-08-02 16:19:36-04:00     2023-06-30   
3    [MGM]  789570  2023-05-01 16:47:31-04:00     2023-03-31   
4    [MGM]  789570  2023-02-24 08:11:14-05:00     2022-12-31   
5    [MGM]  789570  2022-11-02 16:42:58-04:00     2022-09-30   
6    [MGM]  789570  2022-08-03 16:45:51-04:00     2022-06-30   
7    [MGM]  789570  2022-05-02 16:45:11-04:00     2022-03-31   
8    [MGM]  789570  2022-02-25 08:14:02-05:00     2021-12-31   
9    [MGM]  789570  2021-11-03 16:46:09-04:00     2021-09-30   
10   [MGM]  789570  2021-08-06 09:01:31-04:00     2021-06-30   
11   [MGM]  789570  2021-05-03 08:15:45-04:00     2021-03-31   
12   [MGM]  789570  2021-02-26 08:16:34-05:00     2020-12-31   
13   [MGM]  789570  2020-11-03 08:31:44-05:00     2020-09-30   
14   [MGM]  789570  2020-08-03 08:00:58-

   tickers     cik                 reportedAt periodOfReport  \
0    [MGM]  789570  2024-02-23 08:28:47-05:00     2023-12-31   
1    [MGM]  789570  2023-11-08 16:20:52-05:00     2023-09-30   
2    [MGM]  789570  2023-08-02 16:19:36-04:00     2023-06-30   
3    [MGM]  789570  2023-05-01 16:47:31-04:00     2023-03-31   
4    [MGM]  789570  2023-02-24 08:11:14-05:00     2022-12-31   
5    [MGM]  789570  2022-11-02 16:42:58-04:00     2022-09-30   
6    [MGM]  789570  2022-08-03 16:45:51-04:00     2022-06-30   
7    [MGM]  789570  2022-05-02 16:45:11-04:00     2022-03-31   
8    [MGM]  789570  2022-02-25 08:14:02-05:00     2021-12-31   
9    [MGM]  789570  2021-11-03 16:46:09-04:00     2021-09-30   
10   [MGM]  789570  2021-08-06 09:01:31-04:00     2021-06-30   
11   [MGM]  789570  2021-05-03 08:15:45-04:00     2021-03-31   
12   [MGM]  789570  2021-02-26 08:16:34-05:00     2020-12-31   
13   [MGM]  789570  2020-11-03 08:31:44-05:00     2020-09-30   
14   [MGM]  789570  2020-08-03 08:00:58-

   tickers     cik                 reportedAt periodOfReport  \
0    [MGM]  789570  2024-02-23 08:28:47-05:00     2023-12-31   
1    [MGM]  789570  2023-11-08 16:20:52-05:00     2023-09-30   
2    [MGM]  789570  2023-08-02 16:19:36-04:00     2023-06-30   
3    [MGM]  789570  2023-05-01 16:47:31-04:00     2023-03-31   
4    [MGM]  789570  2023-02-24 08:11:14-05:00     2022-12-31   
5    [MGM]  789570  2022-11-02 16:42:58-04:00     2022-09-30   
6    [MGM]  789570  2022-08-03 16:45:51-04:00     2022-06-30   
7    [MGM]  789570  2022-05-02 16:45:11-04:00     2022-03-31   
8    [MGM]  789570  2022-02-25 08:14:02-05:00     2021-12-31   
9    [MGM]  789570  2021-11-03 16:46:09-04:00     2021-09-30   
10   [MGM]  789570  2021-08-06 09:01:31-04:00     2021-06-30   
11   [MGM]  789570  2021-05-03 08:15:45-04:00     2021-03-31   
12   [MGM]  789570  2021-02-26 08:16:34-05:00     2020-12-31   
13   [MGM]  789570  2020-11-03 08:31:44-05:00     2020-09-30   
14   [MGM]  789570  2020-08-03 08:00:58-

Exception: API error: 429 - {"status":429,"error":"You send a lot of requests. We like that. But you exceeded the free query limit of 100 requests. Upgrade your account to get unlimited access. Visit sec-api.io for more."}

In [ ]:
shares_company_lookup_short.head()

In [ ]:
tickers = ['AAPL', 'GOOGL', 'MSFT']  # Example tickers

# Define the start and end dates for your data retrieval
start_date = '2023-01-01'
end_date = '2024-01-01'

# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

# Iterate through each ticker
for ticker in tickers:
    # Fetch historical data for the ticker
    ticker_data = yf.download(ticker, start=start_date, end=end_date)
    
    # Calculate the rolling 30-day average share price
    ticker_data['Average 30 day share price'] = ticker_data['Close'].rolling(window='30D').mean()
    
    # Append the ticker data to the result DataFrame
    result_df = pd.concat([result_df, ticker_data], axis=0)

# Display the result DataFrame
print(result_df)


In [ ]:
from sec_api import QueryApi

queryApi = QueryApi(api_key="1136c7de8da814bb09eea22d0cc6768d1493907372dbf5d88faf3bcd5e419422")
#response = floatApi.get_float(ticker='GOOG')
query = {
  "query": { "query_string": {
      "query": "ticker:TSLA AND filedAt:{2020-01-01 TO 2020-12-31} AND formType:\"10-Q\""
    } },
  "from": "0",
  "size": "10",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = queryApi.get_filings(query)

print(filings)


In [ ]:
query = {
  "query": { "query_string": {
      "query": "formType:\"8-K\" AND description:\"9.01\""
    } },
  "from": "0",
  "size": "10",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = queryApi.get_filings(query)


In [ ]:
print(filings)
